# Introduction

*Notebook author and repo contributor*: Jeff Damasco

*Repo creator and contributor*: Hank Hinnefeld

Hank has a couple examples on how to use his drivers, so you should look at those too. I will say that there are a couple of concepts that are not mentioned, like the use of object oriented programming and the importation of various modules. I'll do my best to explain things further so that you understand the why; if there is something that is not clear, please let me know so I can improve this document.

# Purpose

It's always a pain to have to convert a workplace from one language to another, so the purpose of this is to show you the pain-free reality. Our laboratory has used LabVIEW for most of the time it has existed, but it really is time to let go.

# Requirements

Here I'll be using: https://github.com/masonlab/labdrivers (originally written entirely by Hank Hinnefeld).

# Contributing

If you would like to contribute to the repo, you can send a pull request.

# An in-depth guide to controlling the instrument

What we are doing here is equivalent to finding the libraries of various instruments like the SR830 lock-in amplifier, Keithley 2400 sourcemeter, etc. I will use the SR830 module as an example.

Important note: the notebook is written with the assumption that the file is in the root directory of labdrivers, or that labdrivers is accessible via the Python27 directory.

The workflow here is as follows:

#### From the module, import the constructor

In [1]:
from labdrivers.srs import sr830

To explain this line of code, consider the structure of the labdrivers directory:

```
labdrivers
|
|--- docs
|--- example_nbs
+--- labdrivers
     |
     |--- keithley
     |--- ni
     |--- oxford
     |--- quantum design
     +--- srs
          |
          |--- __init__.py
          |--- sr560.py
          +--- sr830.py
```

`from labdrivers.srs` tells Python to search in `labdrivers/labdrivers/srs`.

`import sr830` tells Python to import the class sr830.

This line allows us to construct a Python object that represents the SR830.

#### Given the GPIB address, instantiate a SR830 object

In [3]:
sr830_gpib_address = 8

lock_in = sr830(sr830_gpib_address)

The GPIB address is something you set on the machine itself. Make sure this is identical or else your program will not work.

`lock_in` is simply an instance of the sr830 class. In the example, `lock_in` connects to the SR830 that has the address `8`. In principle, if you have multiple instruments, you only need to import the module once, and you can create as many instances as you need to.

#### Set up the instrument with whatever parameters you need

In [ ]:
lock_in.setFrequency(137.9)
lock_in.setAmplitude(0.100)
lock_in.setInput(0) # corresponds to 'A' input
lock_in.setTimeConst(9) # corresponds to 300 ms

Each instance of an instrument has a variety of functions associated with it. As you would expect of the lock-in amplifier, you (as the experimentalist) should be able to set the amplifier's frequency, its voltage output, and determine how you would like it to take inputs. The dot notation (e.g. `lock_in[dot]setFrequency(137.9)`) tells the object that I called "`lock_in`" to use the function that follows the dot. The `setFrequency` method also takes in a value, which is the frequency (in Hertz) that you would typically set by hand. There should be a wide variety of functions (the obviously-named "getters" and "setters") to help control your instrument; these functions are found in the class definitions.

Back to the example. The numbers in the frequency and amplitude setters are more or less obvious. The frequency is set in Hertz, and the amplitude refers to the output voltage amplitude, which is in Volts.

However, some of the parameters are not entirely obvious, and so you should look through the Python file (for the SR830: https://github.com/masonlab/labdrivers/blob/master/labdrivers/srs/sr830.py) to figure out if some parameters are coded, like in the time constants.

If you need to investigate the inner workings of other instrument modules, you should be able to look through the directory and find the proper class file.

#### Take a piece of data

In [ ]:
x_data = lock_in.getSinglePoint(1)
theta_data = lock_in.getSinglePoint(4)

Here are a couple more examples of a defined instance of an `sr830` object using a function to perform an action. In this case, the action is to obtain some output data. The parameters can be found in the class file, of course, but to avoid the need of looking it up, the `1` input indicates you want to observe the `X` output of the data, whereas the `4` input indicates you want to observe the `Theta` output of the data.

# Data storage and manipulation

Here is a small walkthrough on how to use the pandas package for creating and modifying data tables.

#### Importing some critical libraries

In [37]:
import pandas as pd
import numpy as np
from __future__ import division

Some notes:

1. The first two lines are import statements without `from`s. The previous `import` statements were used in conjunction with a `from` statement, but this is something that is a bit out of the scope of this notebook. You can ask me if you're curious, though.

2. On the first two lines, I create commonly accepted aliases for `pandas` and `numpy`. While it is completely fine to write out `pandas.something` or `numpy.something` every time, it can get annoying, so to increase readability of your code and to save a miniscule amount of time, it is common to write out `pd.something` or `np.something` instead.

3. Line 3 is not necessary, but I will explain why I imported these. The `__future__` module brings `Python 3.x` functionality to `Python 2.7.x`. In short, `Python 3.x` scripts treat division true division instead of a floor function.

I'm going to begin by establishing a pandas DataFrame instance, which is where all the data will come from.

In [32]:
columns = ['Gate Voltage (V)','Bias Voltage (mV)','dI/dV (2e^2/h)']
data = pd.DataFrame(columns=columns)

I created a list of columns knowing what exactly I want to have in my data set. I will have two inputs, the gate voltage and bias voltage, and I expect that I'll have the differential conductance (dI/dV) as the data I'm measuring.

Let's suppose that I have a new row in my data that describes the differential conductance when the gate voltage is -5.35 V and the voltage bias across the sample is 2.45 mV. And let's suppose that the measurement was found to yield 0.7 2e^2/h of conductance. I'm going to create a new record as a DataFrame for reasons that will be slightly more obvious very soon.

In [33]:
new_record = pd.DataFrame(np.array([[-5.35, 2.45, 0.7]]),
                         columns=columns)

Prior to adding in a new record, the data table looks like this:

In [34]:
data.head()

,Gate Voltage (V),Bias Voltage (mV),dI/dV (2e^2/h)


N.b. `head` is a function which allows you to check out the first 10 rows of a table. You can actually specify the amount of rows as an argument, but remember that the output on a notebook or a console typically is not more than low 10s of lines.

Here's how to insert a new record:

In [35]:
data = data.append(new_record, ignore_index=True)

The `append` method merely outputs a new DataFrame object, but it does not immediately replace the original. Thus, you must set the DataFrame of your data set to be the output of the appending method. This method takes the following arguments: another DataFrame and whether or not the DataFrame ignores the differences in indices between the DataFrame to be appended and the DataFrame that is appending.

Here we assume that the data to append `new_record` already has the same columns as the data set `data`. If not, then you may have records with `NaN` values, and you will require some extra post-processing.

The effect of the `ignore_index` being set to `True` is not obvious here, but consider the case when the data set `data` has `n` rows; `data` will have indices `0, 1, ... , n`, and appending without ignoring the index will force `data` to have indices `0, 1, ..., n, 0`. In practice, this actually will not affect much, but if you are, for whatever reason, performing data operations that call by the index location, then you might run into a runtime error or a 'compile time' error (noting that Python doesn't actually compile; it interprets). As a suggestion, try to keep data organized and ignore the indices of new records.

That said, here is what the data set `data` looks like after appending a new record:

In [36]:
data.head()

,Gate Voltage (V),Bias Voltage (mV),dI/dV (2e^2/h)
0,-5.35,2.45,0.7


After you gather all of your data, then you may output the file. Typically the files will be in a comma-separated value (CSV) format, but there are other file types that you could choose from (refer to the Pandas documentation).

Here's how to do it:

In [ ]:
data.to_csv('output_path.csv', sep='\t', index=False)

Here I am not outputting a file to the Python interpreter directly, so I do not need to write `data = data.to_csv...`. Instead, the output is directly to the directory specified. In this example, only the file name is given, meaning that the file will be called `output_path.csv` and will be placed in the same directory as this notebook. The parameter `sep` specifies the type of separation that will exist between entries, and the parameter `index` specifies if the indices on the DataFrame should be output as a column with the expected data. For our purposes that is generally undesired (unless you would like to follow the order of data acquisition) so that is set to `False`.

# Automating data acquisition

Knowing all of this information, you should be able to automate data acquisition with typical flow control elements:

* `for`
* `while`

The documentation for these two flow control statements are easily searchable and will not be covered here, but note that how you want to control when to exit a loop is completely up to you. To wit, `for` is usually used when you have a very specific range of values that you must loop over. On the other hand, `while` is typically suited for applications where you are waiting for a certain condition (e.g. when the resistance reaches zero, when the conductance is one quantum of conductance, etc.).

#### When to output data

If you would like to output data as one large mass, that should be done after and outside the flow control loop of choice. However, the main disadvantage here is that if you interrupt the program, you lose the chance to save your data. Thus, it's typically a better choice to output data after each loop iteration. Considering that, you might want to think about using the `with...as...` format, as that is usually faster than using `df.to_csv`. If you consider the number of records in your data, if you have an extremely large data set, then each time that you call `df.to_csv` will be more than the previous time, and that is actually quite inefficient. The preferred method is described in the next subsection.

#### with...as...

Here we import another future module:

In [38]:
from __future__ import with_statement

I won't go in-depth about the internal structure (i.e. the "how") of the `with` statement, but basically it allows for the setting up and tearing down of something even if the contents of the `with` statement do not necessarily work. Of course this raises the question of what exactly goes in the `with` statement. For us, it is placing the data in the file. The file is the thing that we will be setting up and tearing down (mentioned a couple sentences ago).

Here is an example of how it works:

In [ ]:
with open('conductance_list.txt') as f:
    
    for voltage in gate_voltage:
        
        lockin_v = lockin.getSinglePoint(1)
        conductance = calculate_conductance(lockin_v)
        f.write(str(voltage) + ',' + str(conductance))

Translating into English, this reads:

1. We open a file called `conductance_list.txt` and alias it as `f`

2. For every value (which we call the variable `voltage`) in the list called `gate_voltage`, perform the following:

    * Get the voltage from the lock-in amplifier object
    
    * Calculate the sample conductance based on that voltage
    
    * Write a line in the file `f` that contains the current gate voltage value and the calculated conductance, separated by a comma

# Mapping out data

For this part, I would like to introduce the use of `matplotlib` and `seaborn` as a method to obtain pretty plots. It's the author's personal opinion that Python could be used to obtain beautiful, customizable plots, though programs such as OriginPro has its uses in quick examinations of data or the exporation of plots with color schemes that conform to whatever standards within a journal article.

The two modules that are important here are:

In [ ]:
import matplotlib
import seaborn as sns